In [1]:
import cv2
import numpy as np
from statistics import median

In [2]:
vidnum = 1

history = 5
persistance = 0
maxPersistance = 10

BlurTuple = (5,5)

CannyLow = 5
CannyHigh = 100

isProbabilistic = False
HoughBinsRho = 1
HoughBinsTheta = np.pi/180
HoughThresholdVotes = 65
HoughThresholdVotesP = 10
MinLineLength = 50
MaxLineGap = 5

MedianLineColor = (0, 0, 255)
MedianLineThickness = 3

rhoRegularisation = 0.1
thetaRegularisation = 0.3

rhoDeviation = 60
thetaDeviation = 0.2
rhoByThetaDev = rhoDeviation/thetaDeviation

pastRhos = [0]
pastThetas = [0]
pastRhoMedian = 0
pastThetaMedian = 0

#Y1s are smaller and
pastYMins = [0]
pastYMaxs = [0]
pastY1MinMedian = 0
pastY2MaxMedian = 0

pastYs = [0]
pastYPersistance = 70

In [3]:
vid = cv2.VideoCapture("./Videos/" + str(vidnum) + ".mp4")

print("Video dimensions are: %d x %d " %(int(vid.get(3)), int(vid.get(4))))

Video dimensions are: 1920 x 1080 


In [4]:
backSubKNN = cv2.createBackgroundSubtractorKNN()
backSubMOG2 = cv2.createBackgroundSubtractorMOG2()
backSubMOG = cv2.bgsegm.createBackgroundSubtractorMOG()
backSubGMG = cv2.bgsegm.createBackgroundSubtractorGMG()
backSubCNT = cv2.bgsegm.createBackgroundSubtractorCNT()
backSubGSOC = cv2.bgsegm.createBackgroundSubtractorGSOC()
backSubLSBP = cv2.bgsegm.createBackgroundSubtractorLSBP()
# backSubGMG = cv2.cuda.createBackgroundSubtractorGMG()
# backSubFGD = cv2.cuda.createBackgroundSubtractorFGD()
# backSubMOG = cv2.cuda.createBackgroundSubtractorMOG()

# backSubMOG.setHistory(50)


# backSubKNN.setHistory(10)
backSubKNN.setDetectShadows(False)

In [5]:
frame_width=1920
frame_height=1080

cap = cv2.VideoCapture("./Videos/" + str(vidnum) + ".mp4")

# Get the Default resolutions
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

# Define the codec and filename.
out = cv2.VideoWriter('output.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 10, (frame_width,frame_height))
 
# fourcc = cv2.VideoWriter_fourcc(*'8BPS')
# out = cv2.VideoWriter("./outt" + str(vidnum) + ".avi",fourcc, 10, (frame_width,frame_height), True)

In [6]:
def findRangeY(linesP) :
    global pastYMins, pastYMaxs, pastYMinMedian, pastYMaxMedian
    #ansmin, ansmax = 0, 0
    if linesP is not None:
        for line in linesP:
            x1,y1,x2,y2 = line[0]
            currentYMin = 0
            currentYMax = 0
            if y1 >= y2:
                pastYMins.append(y2)
                pastYMaxs.append(y1)
            else:
                pastYMins.append(y1)
                pastYMaxs.append(y2)
            
            pastYs.append(y1)
            pastYs.append(y2)
            
#             if y1>ansmax :
#                 ansmax = y1
#             if y2>ansmax :
#                 ansmax = y2
#             if y1<ansmin :
#                 ansmin = y1
#             if y2<ansmin :
#                 ansmin = y2
            if len(pastYs)>pastYPersistance :
                del pastYs[0]
                del pastYs[0]
                
            if len(pastYMaxs)>pastYPersistance:
                del pastYMaxs[0]
            if len(pastYMins)>pastYPersistance:
                del pastYMins[0]
            
    return max(pastYs), min(pastYs)
#     return int(median(pastYMaxs)), int(median(pastYMins))


    




def giveEndPoints(rho, theta, linesP):
    global pastY1s, pastY2s, pastY1Median, pastY2Median
    a = np.cos(theta)
    b = np.sin(theta)
    x0 = a*rho
    y0 = b*rho
    maxLength = 2000
    y1, y2 = findRangeY(linesP)
    lengthmax = (y1 - y0)/a
    lengthmin = (y2 - y0)/a

    x1 = int(x0 + lengthmax*(-b))
    x2 = int(x0 + lengthmin*(-b))
#     x1 = int(x0 + maxLength*(-b))
#     y1 = int(y0 + maxLength*(a))
#     x2 = int(x0 - maxLength*(-b))
#     y2 = int(y0 - maxLength*(a))
    return (x1,y1,x2,y2)
    
    
def getMean(lines):
    tempx, tempy = (0,0)
    if lines is not None:
        for line in lines:
            rho, theta = line[0]
            tempx, tempy = tempx + rho, tempy + theta
        tempx, tempy = tempx/len(lines), tempy/len(lines)
    return tempx, tempy




def getMedian(lines):
    global pastRhoMedian, pastThetaMedian, pastRhos, pastThetas, persistance
    rhoList = []
    thetaList = []
    rhoMedian, thetaMedian = 0, 0
    
    if lines is not None:
        for line in lines:
            rho, theta = line[0]
            rhoList.append(rho)
            thetaList.append(theta)
        rhoMedian = median(rhoList)
        thetaMedian = median(thetaList)
    
#         print('\t\t', rhoMedian, thetaMedian, pastRhoMedian-rhoMedian + (pastThetaMedian-thetaMedian)*rhoByThetaDev, persistance, (pastRhoMedian==0 or pastThetaMedian==0 or len(pastRhos)<history, abs(pastRhoMedian-rhoMedian + (pastThetaMedian-thetaMedian)*rhoByThetaDev)<rhoDeviation, persistance >= maxPersistance))
    
        if pastRhoMedian==0 or pastThetaMedian==0 or len(pastRhos)<history:
            pastRhos.append(rhoMedian)
            pastThetas.append(thetaMedian)
#         elif abs(pastRhoMedian-rhoMedian)/pastRhoMedian<rhoRegularisation and abs(pastThetaMedian-thetaMedian)/pastThetaMedian<thetaRegularisation:
        elif abs(pastRhoMedian-rhoMedian + (pastThetaMedian-thetaMedian)*rhoByThetaDev)<rhoDeviation:
            pastRhos.append(rhoMedian)
            pastThetas.append(thetaMedian)
        elif persistance >= maxPersistance:
            pastRhos = [rhoMedian]
            pastThetas = [thetaMedian]
            persistance = 0
        else:
            persistance += 1
#             print('yo!, them persistance be:', persistance)
#         print('\t\t', rhoMedian, thetaMedian, pastRhoMedian-rhoMedian + (pastThetaMedian-thetaMedian)*rhoByThetaDev, persistance, (pastRhoMedian==0 or pastThetaMedian==0 or len(pastRhos)<history, abs(pastRhoMedian-rhoMedian + (pastThetaMedian-thetaMedian)*rhoByThetaDev)<rhoDeviation, persistance >= maxPersistance))    
        if len(pastRhos) > history:
            del pastRhos[0]
            del pastThetas[0]
        
    rhoMedian, thetaMedian = median(pastRhos), median(pastThetas)
    pastRhoMedian, pastThetaMedian = rhoMedian, thetaMedian
#     print(pastRhos, pastThetas, '\n', pastRhoMedian, pastThetaMedian, '\n',  end='\n\n')
    return rhoMedian, thetaMedian

In [7]:
def processFrame(frame):
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    low_ring = np.array([10, 80, 50])
    high_ring = np.array([55,220, 180])
    mask = cv2.inRange(hsv, low_ring, high_ring)
    
    frame[np.where((mask==255))] = [30,40,20]
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    subbedFrameMOG = backSubMOG.apply(gray)
    
    cv2.imshow('subbedFrame', subbedFrameMOG)
    blurMOG = cv2.medianBlur(subbedFrameMOG, 9)
    smallBlurMOG = cv2.resize(blurMOG, (800,600), interpolation=cv2.INTER_LINEAR)
    
    return blurMOG
    

def processVideo():
    while vid.isOpened():
        ret, frame = vid.read()
        if ret==True:
            copy = frame.copy()
            
            
#             print(mask)
#             cv2.imshow('mask', mask)
#             frame.setTo([20,40,30], mask)
      
    
#             for i in range(len(frame)):
#                 for j in range(len(frame[0])):
#                     if mask[i][j]==1:
#                         frame[i][j] = [20,40,30]
            
                        
#             cv2.imshow('gray', gray)
#             blur = cv2.blur(gray, BlurTuple)
#             subbedFrameMOGnoBlur = backSubMOG.apply(gray)
            
        
            
#             smallSub = cv2.resize(subbedFrameMOG, (800,600), interpolation=cv2.INTER_LINEAR)

#             cv2.imshow('noBlur', smallSub)
#             cv2.imshow('blurSub', smallBlurMOG)
#             subbedFrameMOG2 = backSubMOG2.apply(blur)
#             subbedFrameMOG = backSubMOG.apply(frame)
    #         subbedFrameGMG = backSubGMG.apply(frame)
            blurredFrame = processFrame(frame)
            edges = cv2.Canny(blurredFrame, CannyLow, CannyHigh)
            
#             out.write(subbedFrameMOG)
            out.write(blurredFrame) 

            
#             if isProbabilistic:
#                 linesP = cv2.HoughLinesP(edges, HoughBinsRho, HoughBinsTheta, HoughThresholdVotesP, minLineLength=MinLineLength, maxLineGap=MaxLineGap)
#             else:
#                 lines = cv2.HoughLines(edges, HoughBinsRho, HoughBinsTheta, HoughThresholdVotes)
#             print(lines)
            colorMOG = cv2.cvtColor(blurredFrame, cv2.COLOR_BAYER_GR2BGR)
    
            lines = cv2.HoughLines(edges, HoughBinsRho, HoughBinsTheta, HoughThresholdVotes)
            linesP = cv2.HoughLinesP(edges, HoughBinsRho, HoughBinsTheta, HoughThresholdVotesP, minLineLength=MinLineLength, maxLineGap=MaxLineGap)
#             print(lines)


            rho, theta = getMedian(lines)
            x1, y1, x2, y2 = giveEndPoints(rho, theta, linesP)
            cv2.line(frame, (x1, y1), (x2, y2), MedianLineColor, MedianLineThickness)
    
#             if lines is not None:
#                 for line in lines:
#                     rho,theta = line[0]
#                     x1, y1, x2, y2 = giveEndPoints(rho, theta, linesP)
#                     cv2.line(frame, (x1, y1), (x2, y2), (0,255,0), 1)
                    
            
    
#             if linesP is not None:
#                 for line in linesP:
#                     x1, y1, x2, y2 = line[0]                   
#                     cv2.line(frame, (x1, y1), (x2, y2), (0,0,255), MedianLineThickness)
                       

#             x1, y1, x2, y2 = giveEndPoints(-1030., 2.76234, linesP)
#             cv2.line(frame, (x1, y1), (x2, y2), MedianLineColor, MedianLineThickness)
            
#             x1, y1, x2, y2 = giveEndPoints(-1090., 2.96234, linesP)
#             cv2.line(frame, (x1, y1), (x2, y2), MedianLineColor, MedianLineThickness)
        
        
    
            smallFrame = cv2.resize(frame, (800,600), interpolation=cv2.INTER_LINEAR)
            cv2.imshow('original', smallFrame)
    #         cv2.imshow('MOG2', subbedFrameMOG2)
#             cv2.imshow('KNN', colorKNN)
    #         cv2.imshow('edges', edges)         
    #         cv2.imshow('blur', blur)         


    #         cv2.imshow('GMG', subbedFrameGMG)
#             cv2.imshow('MOG', subbedFrameMOG)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        else:
            break
    vid.release()
    out.release()
    cv2.destroyAllWindows()

In [8]:
processVideo()

KeyboardInterrupt: 

In [ ]:
out.release() 
out = None
cv2.destroyAllWindows()


rhoVariation = 60 total, 45 on each side

tempLines = [[[-1030.            2.86234  ]]

 [[-1032.            2.86234  ]]

 [[-1043.            2.8797932]]

 [[-1053.            2.8797932]]

 [[-1055.            2.8797932]]

 [[-1058.            2.8972466]]

 [[-1040.            2.8797932]]

 [[-1019.            2.8448865]]

 [[-1021.            2.8448865]]

 [[-1023.            2.8448865]]

 [[-1072.            2.9146998]]

 [[-1025.            2.8448865]]

 [[-1017.            2.8448865]]

 [[-1016.            2.8274333]]

 [[-1061.            2.8972466]]

 [[-1085.            2.9321532]]

 [[-1010.            2.8274333]]

 [[-1014.            2.8274333]]

 [[-1037.            2.8448865]]

 [[-1064.            2.9146998]]

 [[ -994.            2.8099802]]

 [[-1005.            2.8274333]]

 [[-1002.            2.8099802]]

 [[-1028.            2.8448865]]

 [[-1088.            2.9321532]]

 [[-1008.            2.8274333]]

 [[-1098.            2.9496064]]]